In [58]:
import numpy as np

In [59]:
def gaussian_formal(x,y,x_0=0,y_0=0,sigx=1,sigy=1,A=1):
    return A*np.exp(-(((x-x_0)**2)/(2*sigx**2)+((y-y_0)**2)/(2*sigy**2)))

In [60]:
def gaussian(d,t):
    d=d*t
    return np.exp(-(d**2))

In [61]:
def mexican_hat(d,t):
    d=d*t
    return (2-4*d**2)*np.exp(-(d**2))

In [62]:
def exp_res(t,lambd):
    return np.exp(-t/lambd)

In [63]:
class Kohen:
    def __init__(self,M,N):
        self.Neurons=[[0]*N for x in range(M)]
        
    def euclidean(x,y):
        return np.sqrt(np.sum((x-y)**2))
    
    def train(self,Data,lambd,neighbour_fun,learning_res,scale_x=1,scale_y=1):
        #initaite with random vectors
        for x in len(self.Neurons):
            for y in range(len(self.Neurons[0])):
                self.Neurons[x][y]=np.random.random(Data.shape[1])*2-1
                
        for s in range(1,lambd):
            for t in range(Data.shape[0]):
                #find BMU
                D=Data[t]
                BMU_index=[0,0]
                BMU_value=self.euclidean(self.Neurons[BMU_index[0]][BMU_index[1]],D)
                for x in len(self.Neurons):
                    for y in range(len(self.Neurons[0])):
                        D_t=self.euclidean(self.Neurons[x][y],D)
                        if D_t<BMU_value:
                            BMU_value=D_t
                            BMU_index[0]=x
                            BMU_index[1]=y
                #update
                for x in len(self.Neurons):
                    for y in range(len(self.Neurons[0])):
                        d=np.sqrt((scale_x*(x-BMU_index))**2+(scale_y*(y-BMU_index))**2)
                        self.Neurons[x][y]=self.Neurons[x][y]+neighbour_fun(d,s)*learning_res(s,lambd)*(D-self.Neurons[x][y])
                
        

In [64]:
Map_test=Kohen(30,40)